# Replicating Mitra and Gilbert (2014) Using `pandas`, `statsmodels`, and `sklearn`

[Tanushree Mitra](http://www.cc.gatech.edu/~tmitra3/) and [Eric Gilbert](http://eegilbert.org/) published a paper at CSCW 2014 called _[The Language that Gets People to Give: Phrases that Predict Success on Kickstarter](https://comp.social.gatech.edu/papers/cscw14.crowdfunding.mitra.pdf)_. It's a very interesting paper that uses text analysis to find phrases that predict funding success on the crowdfunding site [Kickstarter](https://www.kickstarter.com/). I thought it would be interesting to apply their methods to a different type of crowdfunding site, [DonorsChoose](https://www.donorschoose.org/), where people can donate to teachers to support classroom projects.

In addition to applying their method to a new dataset, I thought I'd do so in a jupyter notebook and walk through some of the Python libraries I'm using to do this project. In doing so, I hope to demonstrate the utility of scientific Python libraries to researchers in HCI and related fields. My intended audience here is people who have some background in Python and have possibly fit a model or two in R before, but don't know a lot about the scientific Python landscape. My goal is to introduce people to loading data in `pandas`, fitting models in `statsmodels`, and doing some basic machine learning with `sklearn`. I'm also assuming you read the paper and mostly understood it, so I won't go into much detail about what things like _bag of words_ representation means, but I'll link to articles if you need a refresher.

If you're new to scientific computing in Python, I recommend you check out the [Anaconda](https://www.continuum.io/downloads) distribution, which will get you all the libraries that are used below. It will also install jupyter, which means you can [clone this notebook](https://github.com/matthewheston/matthewheston.github.io/tree/master/etc/donorschoose), open it up on your machine, and run all this code at home.

_Note_: While I wanted to make this a jupyter notebook with the intention of you being able to easily clone and run on your own machine, in practice, you're going to need a lot of RAM to run some of the models. I was still able to run the sections up to the `sklearn` section on my laptop.

## pandas

[pandas](http://pandas.pydata.org/) describes itself as "an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language." We'll be storing all of our data in pandas `DataFrame` objects. It's convention to import this library as `pd`. We also use the `set_option` function to set the number of maximum columns displayed when we peek at a `DataFrame`. We'll set this to a high number so things don't get truncated when we look at them.

In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)

Kaggle sponsored a contest with DonorsChoose data a couple years ago, so we'll just use that data that's already been collected. You can read more about the data [on their website](https://www.kaggle.com/c/kdd-cup-2014-predicting-excitement-at-donors-choose/data). They provide several CSV files. The first has general information about projects. The second has outcome variables (including whether or not a project reached its funding goal), and the last includes both short descriptions as well as the longer essays teachers upload with their projects (which is what we'll be analyzing). pandas offers a nice `read_csv` method to read csv files into a `DataFrame`.

In [2]:
projects = pd.read_csv("projects.csv")
outcomes = pd.read_csv("outcomes.csv")
essays = pd.read_csv("essays.csv")

You can do a lot of different things with DataFrames. To start, we'll just use the `head` function to take a look at the first 5 rows of each one.

In [3]:
projects.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,school_district,school_county,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_prefix,teacher_teach_for_america,teacher_ny_teaching_fellow,primary_focus_subject,primary_focus_area,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted
0,316ed8fb3b81402ff6ac8f721bb31192,42d43fa6f37314365d08692e08680973,c0e6ce89b244764085691a1b8e28cb81,6.362701e+10,36.576340,-119.608713,Selma,CA,93662,NaN,Selma Unified Sch District,Fresno,f,f,f,f,f,f,Mrs.,f,f,Literature & Writing,Literacy & Language,College & Career Prep,Applied Learning,Books,highest poverty,Grades 6-8,30,555.81,653.89,32,f,f,2014-05-12
1,90de744e368a7e4883223ca49318ae30,864eb466462bf704bf7a16a585ef296a,d711e47810900c96f26a5d0be30c446d,4.837020e+11,32.911179,-96.723640,Dallas,TX,75243,urban,Richardson Ind School District,Dallas,f,f,f,f,f,f,Mrs.,f,f,Literacy,Literacy & Language,ESL,Literacy & Language,Books,highest poverty,Grades PreK-2,30,296.47,348.79,22,f,f,2014-05-12
2,32943bb1063267de6ed19fc0ceb4b9a7,37f85135259ece793213aca9d8765542,665c3613013ba0a66e3a2a26b89f1b68,4.103270e+11,45.166039,-122.414576,Colton,OR,97017,rural,Colton School District 53,Clackamas,f,f,f,f,f,f,Mr.,f,f,Literacy,Literacy & Language,Mathematics,Math & Science,Technology,high poverty,Grades PreK-2,30,430.89,506.93,17,f,f,2014-05-11
3,bb18f409abda2f264d5acda8cab577a9,2133fc46f951f1e7d60645b0f9e48a6c,4f12c3fa0c1cce823c7ba1df57e90ccb,3.600153e+11,40.641727,-73.965655,Brooklyn,NY,11226,urban,New York City Dept Of Ed,Kings (Brooklyn),f,t,f,f,f,f,Mr.,t,f,Social Sciences,History & Civics,Special Needs,Special Needs,Books,highest poverty,Grades 3-5,30,576.07,677.73,12,f,f,2014-05-11
4,24761b686e18e5eace634607acbcc19f,867ff478a63f5457eaf41049536c47cd,10179fd362d7b8cf0e89baa1ca3025bb,6.227100e+10,34.043939,-118.288371,Los Angeles,CA,90006,urban,Los Angeles Unif Sch Dist,Los Angeles,f,f,f,f,f,f,Ms.,f,f,Mathematics,Math & Science,Literacy,Literacy & Language,Other,highest poverty,Grades PreK-2,30,408.40,480.47,24,f,f,2014-05-11


In [4]:
outcomes.head()

,projectid,is_exciting,at_least_1_teacher_referred_donor,fully_funded,at_least_1_green_donation,great_chat,three_or_more_non_teacher_referred_donors,one_non_teacher_referred_donor_giving_100_plus,donation_from_thoughtful_donor,great_messages_proportion,teacher_referred_count,non_teacher_referred_count
0,ffffc4f85b60efc5b52347df489d0238,f,NaN,f,NaN,f,NaN,NaN,NaN,NaN,NaN,NaN
1,ffffac55ee02a49d1abc87ba6fc61135,f,f,t,t,f,t,f,f,57,0,7
2,ffff97ed93720407d70a2787475932b0,f,f,t,t,t,t,t,f,100,0,3
3,ffff418bb42fad24347527ad96100f81,f,f,f,t,t,f,f,f,100,0,1
4,ffff2d9c769c8fb5335e949c615425eb,t,t,t,t,t,f,t,f,63,6,2


In [5]:
essays.head()

,projectid,teacher_acctid,title,short_description,need_statement,essay
0,ffffc4f85b60efc5b52347df489d0238,c24011b20fc161ed02248e85beb59a90,iMath,It is imperative that teachers bring technolog...,My students need four iPods.,I am a fourth year fifth grade math teacher. T...
1,ffffac55ee02a49d1abc87ba6fc61135,947066d0af47e0566f334566553dd6a6,Recording Rockin' Readers,Can you imagine having to translate everything...,My students need a camcorder.,Can you imagine having to translate everything...
2,ffff97ed93720407d70a2787475932b0,462270f5d5c212162fcab11afa2623cb,Kindergarten In Need of Important Materials!,It takes a special person to donate to a group...,My students need 17 assorted classroom materia...,Hi. I teach a wonderful group of 4-5 year old ...
3,ffff7266778f71242675416e600b94e1,b9a8f14199e0d8109200ece179281f4f,Let's Find Out!,My Kindergarten students come from a variety o...,"My students need 25 copies of Scholastic's ""Le...",My Kindergarten students come from a variety o...
4,ffff418bb42fad24347527ad96100f81,e885fb002a1d0d39aaed9d21a7683549,Whistle While We Work!,"By using the cross curricular games requested,...",My students need grade level appropriate games...,All work and no play makes school a dull place...


For simplicity of working with this data, we'll put everything in one big `DataFrame`. If you've ever worked with SQL before, you can think of this as joining these three tables on the `projectid` column. To do that, we'll use the `merge` function on the different DataFrame objects.

_Note_: Sometimes you'll be joining two DataFrames that have some of the same column names. To deal with this, pandas will append a suffix to these column names so you can distinguish which original DataFrame a column came from. That's what the suffixes argument does below. This doesn't really matter for what we're doing, but it's something to keep in mind.

In [6]:
projects_and_outcomes = projects.merge(outcomes, on="projectid", suffixes=('',''))

In [7]:
all_projects = projects_and_outcomes.merge(essays, on="projectid", suffixes=('','_'))

To make sure everything worked, we'll take a glance at our new DataFrame that contains everything.

In [8]:
all_projects.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,school_district,school_county,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_prefix,teacher_teach_for_america,teacher_ny_teaching_fellow,primary_focus_subject,primary_focus_area,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted,is_exciting,at_least_1_teacher_referred_donor,fully_funded,at_least_1_green_donation,great_chat,three_or_more_non_teacher_referred_donors,one_non_teacher_referred_donor_giving_100_plus,donation_from_thoughtful_donor,great_messages_proportion,teacher_referred_count,non_teacher_referred_count,teacher_acctid_,title,short_description,need_statement,essay
0,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103,suburban,Elgin School District U-46,Du Page,f,f,f,f,f,f,Mrs.,f,f,Special Needs,Special Needs,Literacy,Literacy & Language,Other,moderate poverty,Grades 3-5,30,444.36,522.78,7,f,f,2013-12-31,f,f,t,f,t,t,t,f,80,0,6,ebc7c90b6c92a069432e0714b8d93dfd,Help Us Make All of the Pieces Fit,"If they can't learn the way we teach, we teach...","My students need puzzles, games and visual lea...","If they can't learn the way we teach, we teach..."
1,33d59ac771b80222ad63ef0f4ac47ade,de83b4c1f6428a15032c207c1d5e572a,d91a805b213bf74ae77b94e0de2b73ad,1.601530e+11,43.501154,-112.056780,Idaho Falls,ID,83402,urban,Idaho Falls School District 91,Bonneville,f,f,f,f,f,f,Mrs.,f,f,Mathematics,Math & Science,NaN,NaN,Supplies,high poverty,Grades 3-5,30,233.24,274.40,30,f,f,2013-12-31,f,NaN,f,NaN,f,NaN,NaN,NaN,NaN,NaN,NaN,de83b4c1f6428a15032c207c1d5e572a,Capacity...What Is That?,"Which is bigger, three liters or three quarts?...",My students need capacity tools and resources ...,"Which is bigger, three liters or three quarts?..."
2,1a3aaeffc56dd2a421e37d8298024c0a,f4c9ed095b85458dcf858e25f203af00,9310d3eb447a4e46bc5fc31ed007ceac,3.302610e+11,42.888244,-71.320224,Derry,NH,3038,suburban,School Administrative Unit 10,Rockingham,f,f,f,f,f,f,Mrs.,f,f,Environmental Science,Math & Science,Applied Sciences,Math & Science,Technology,moderate poverty,Grades 6-8,30,285.09,335.40,230,f,f,2013-12-31,f,f,f,t,f,f,f,f,NaN,0,2,f4c9ed095b85458dcf858e25f203af00,Catching the Sun!,Do you remember classrooms that used just book...,My students need UV sensitive beads and a Vern...,Do you remember classrooms that used just book...
3,33aa19ee4da4c5adf47d0dfb84fab5ef,17768031eb40de8d4497dbb54df48742,9ac70da58322783f82152eecc140a812,5.103240e+11,37.476158,-77.488397,Richmond,VA,23224,urban,Richmond City School District,Richmond City,f,f,f,f,f,f,Ms.,f,f,Literacy,Literacy & Language,NaN,NaN,Other,highest poverty,Grades PreK-2,30,232.94,274.05,18,f,f,2013-12-31,f,f,f,t,f,f,f,f,NaN,0,1,17768031eb40de8d4497dbb54df48742,Let's Get Organized!,My class was given the beautiful gift of books...,My students need labels and book bins to organ...,My class was given the beautiful gift of books...
4,e31c0ea8b68f404699dfb0d39e9bc99b,0f1bc5b4700fd33383be104442660178,cb9f688cf59e3ee22a087d616ca8f5d7,1.709930e+11,41.952851,-87.650233,Chicago,IL,60613,urban,Ravenswood-ridge Elem Network,Cook,f,t,f,f,f,f,Mr.,f,f,Environmental Science,Math & Science,NaN,NaN,Supplies,highest poverty,Grades 6-8,30,513.41,604.01,70,t,f,2013-12-31,f,f,t,t,f,f,t,f,50,0,2,0f1bc5b4700fd33383be104442660178,Making Earth Interesting,Thinking back in school science was either rea...,My students need an Interactive whiteboard les...,Thinking back in school science was either rea...


### Summarizing Data

You can access columns in DataFrame's a couple different ways. One of the variables we have is `school_metro` which says whether a school is rural, suburban, or urban. To reference this column, we can call either `all_projects.school_metro` or `all_projects["school_metro"]`. I like to use the first one when possible because jupyter is smart enough to put it in auto complete as I'm typing, so I don't have to worry about mistyping a column name.

For nominal variables like `school_metro`, you can call the `value_counts` method, which will show you all of the different levels of the variables and how many times they occur.

In [9]:
all_projects.school_metro.value_counts()

urban       328716
suburban    140808
rural        74314
Name: school_metro, dtype: int64

In [10]:
all_projects.primary_focus_subject.value_counts()

Literacy                 188604
Literature & Writing      76869
Mathematics               74637
Special Needs             39011
Applied Sciences          29085
Visual Arts               28765
Environmental Science     24931
Health & Life Science     23040
Music                     19254
History & Geography       15980
Early Development         13222
Other                     12788
Social Sciences            9639
ESL                        9199
Character Education        9168
Performing Arts            7933
Gym & Fitness              6490
Health & Wellness          5991
College & Career Prep      5952
Foreign Languages          5117
Sports                     3290
Extracurricular            2906
Civics & Government        2331
Economics                  1492
Community Service          1392
Parent Involvement         1217
Nutrition                   987
Name: primary_focus_subject, dtype: int64

For continuous variables like `students_reached` or `total_price_excluding_optional_support`, you can call the `describe` method which will give you some descriptive statistics.

In [11]:
all_projects.students_reached.describe()

count    619182.000000
mean         97.439624
std        2364.069969
min           0.000000
25%          22.000000
50%          30.000000
75%         100.000000
max      999999.000000
Name: students_reached, dtype: float64

You'll notice the max value on students reached is 999,999, which seems like a bit of a stretch. You can use the following syntax in order to make conditional selections in your `DataFrame`. We'll use only projects that have less than 500 students in the `students_reached` variable.

In [12]:
all_students = all_students[all_students.students_reached < 500]

NameError: name 'all_students' is not defined

In [13]:
all_projects.total_price_excluding_optional_support.describe()

count      619326.000000
mean          535.981751
std         13125.958028
min             0.000000
25%           267.000000
50%           410.660000
75%           580.367500
max      10250017.000000
Name: total_price_excluding_optional_support, dtype: float64

## statsmodels

In the Mitra and Gilbert paper, they first fit a logistic regression model using variables such as the amount of money a project was trying to raise and whether or not a video was present to predict whether a project was funded or not. They then use these as control variables in their model with the linguistic featues they extract.

Before getting into text analysis, let's fit a model using the variables we looked at above: `school_metro`, `primary_focus_subject`, `students_reached`, and `total_price_excluding_optional_support`. Not all of the rows in our DataFrame have these values populated. For simplicity, we'll just drop instances that don't have these. First, let's look at how many total projects we have.

In [14]:
len(all_projects)

619326

Next we'll use the `dropna` method to drop rows that don't have these variables populated. We do this by passing a list of column names to check as the `subset` argument to this function. We'll also drop those that don't have an essay, since we'll be working with those later.

In [15]:
all_projects = all_projects.dropna(subset=["school_metro", "primary_focus_subject", 
                                           "essay", "students_reached", "total_price_excluding_optional_support"])

In [16]:
len(all_projects)

543693

This still leaves us with 543,693 projects. Now let's get to fitting a model.

The dependent variable in our model will be `fully_funded`: whether or not a project reached its funding goal. In the CSV provided, this takes on the value of either `f` or `t` for false or true. `statsmodels` will expect this to be represent as 0's and 1's. There are different ways you could do this. We'll use the `LabelEndoder` from the library `sklearn`. The purpose of this class is to transform labels into numeric values, which is exactly what we want to do here. Since 'f' comes before 't' alphabetically, it will automatically assign 'f' to 0 and 't' to 1. We'll store these 0's and 1's in a new column on our DataFrame called `y`. We'll do this by simply assigning `all_projects["y"]` the output of `le.fit_transform(all_projects.fully_funded)`. We'll take a look at the first 5 values of these columns to make sure this worked as we expected.

In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
all_projects["y"] = le.fit_transform(all_projects.fully_funded)
all_projects[["y", "fully_funded"]].head()

,y,fully_funded
0,1,t
1,0,f
2,0,f
3,0,f
4,1,t


If you've ever fit a model in R before, you might be familiar with a model that looks like this:

`y ~ school_metro + primary_focus_subject + students_reached + total_price_excluding_optional_support`

If you aren't familiar, you can see that we define the formula with our dependent variable on the left side of the `~` and the independent variables on the other side. You can do other things like define interactions, but we're just going to do a simple additive model here.

`statsmodels` has some support for R-type formulas, but not for all models, and, importantly for us, not for logistic regression. However, we can use the `patsy` library. `statsmodels` will want a one-dimensional array of our dependent variable, and a matrix of our indepdendent variables. We can get this by specifying the formula above and calling the `dmatrices` function in `patsy`. Also note this automatically takes care of creating dummy variables for our categorical variables.

In [18]:
import patsy
formula = "y ~ school_metro + primary_focus_subject + students_reached + total_price_excluding_optional_support"
y,X = patsy.dmatrices(formula, all_projects, return_type='dataframe')

In [19]:
X.head()

,Intercept,school_metro[T.suburban],school_metro[T.urban],primary_focus_subject[T.Character Education],primary_focus_subject[T.Civics & Government],primary_focus_subject[T.College & Career Prep],primary_focus_subject[T.Community Service],primary_focus_subject[T.ESL],primary_focus_subject[T.Early Development],primary_focus_subject[T.Economics],primary_focus_subject[T.Environmental Science],primary_focus_subject[T.Extracurricular],primary_focus_subject[T.Foreign Languages],primary_focus_subject[T.Gym & Fitness],primary_focus_subject[T.Health & Life Science],primary_focus_subject[T.Health & Wellness],primary_focus_subject[T.History & Geography],primary_focus_subject[T.Literacy],primary_focus_subject[T.Literature & Writing],primary_focus_subject[T.Mathematics],primary_focus_subject[T.Music],primary_focus_subject[T.Nutrition],primary_focus_subject[T.Other],primary_focus_subject[T.Parent Involvement],primary_focus_subject[T.Performing Arts],primary_focus_subject[T.Social Sciences],primary_focus_subject[T.Special Needs],primary_focus_subject[T.Sports],primary_focus_subject[T.Visual Arts],students_reached,total_price_excluding_optional_support
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,7,444.36
1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,30,233.24
2,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,230,285.09
3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,18,232.94
4,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,70,513.41


To fit the model, all we have to do is supply these variables to the Logit class from `statsmodels` and call the `fit` function. We can then print out a nice summary that will look similar to things you may have seen in other stats packages.

In [20]:
from statsmodels.discrete.discrete_model import Logit
model_fit = Logit(y, X).fit()

Optimization terminated successfully.
         Current function value: 0.587256
         Iterations 8


In [21]:
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:               543693
Model:                          Logit   Df Residuals:                   543662
Method:                           MLE   Df Model:                           30
Date:                Mon, 14 Nov 2016   Pseudo R-squ.:                 0.03559
Time:                        09:51:59   Log-Likelihood:            -3.1929e+05
converged:                       True   LL-Null:                   -3.3107e+05
                                        LLR p-value:                     0.000
==================================================================================================================
                                                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------------------------
Intercept                                          1.0462      0.017     62.571      0.000         1.013     1.079
school_metro[T.suburban]                           0.3623      0.010     37.820      0.000         0.344     0.381
school_metro[T.urban]                              0.6648      0.009     76.897      0.000         0.648     0.682
primary_focus_subject[T.Character Education]      -0.2263      0.029     -7.788      0.000        -0.283    -0.169
primary_focus_subject[T.Civics & Government]      -0.1074      0.053     -2.011      0.044        -0.212    -0.003
primary_focus_subject[T.College & Career Prep]    -0.2200      0.034     -6.468      0.000        -0.287    -0.153
primary_focus_subject[T.Community Service]        -0.2307      0.066     -3.508      0.000        -0.360    -0.102
primary_focus_subject[T.ESL]                      -0.4998      0.028    -18.120      0.000        -0.554    -0.446
primary_focus_subject[T.Early Development]        -0.5381      0.025    -21.861      0.000        -0.586    -0.490
primary_focus_subject[T.Economics]                 0.2230      0.069      3.212      0.001         0.087     0.359
primary_focus_subject[T.Environmental Science]     0.2554      0.022     11.407      0.000         0.212     0.299
primary_focus_subject[T.Extracurricular]          -0.2891      0.046     -6.305      0.000        -0.379    -0.199
primary_focus_subject[T.Foreign Languages]        -0.3906      0.036    -10.988      0.000        -0.460    -0.321
primary_focus_subject[T.Gym & Fitness]            -0.3891      0.032    -12.084      0.000        -0.452    -0.326
primary_focus_subject[T.Health & Life Science]     0.0413      0.022      1.861      0.063        -0.002     0.085
primary_focus_subject[T.Health & Wellness]        -0.2557      0.034     -7.448      0.000        -0.323    -0.188
primary_focus_subject[T.History & Geography]      -0.2204      0.024     -9.205      0.000        -0.267    -0.173
primary_focus_subject[T.Literacy]                 -0.3509      0.016    -22.477      0.000        -0.382    -0.320
primary_focus_subject[T.Literature & Writing]     -0.3609      0.017    -21.395      0.000        -0.394    -0.328
primary_focus_subject[T.Mathematics]              -0.2036      0.017    -11.952      0.000        -0.237    -0.170
primary_focus_subject[T.Music]                     0.1433      0.024      6.015      0.000         0.097     0.190
primary_focus_subject[T.Nutrition]                 0.2191      0.089      2.453      0.014         0.044     0.394
primary_focus_subject[T.Other]                    -0.7351      0.025    -29.928      0.000        -0.783    -0.687
primary_focus_subject[T.Parent Involvement]       -0.7092      0.066    -10.682      0.000        -0.839    -0.579
primary_focus_subject[T.Performing Arts]           0.0114      0.031      0.364      0.715        -0.050     0.073
primary_focus_subject

If you aren't familiar with interpreting odds ratios, [this page](http://www.ats.ucla.edu/stat/mult_pkg/faq/general/odds_ratio.htm) does a nice job walking through some examples. I won't spend too much time here going through these results, but I did want to give an example of using `statsmodels` to fit a model.

For the rest of this project, we'll use the `LogisticRegression` class from the `sklearn` library, because it makes things like cross validation and working with our text features (which we'll have a lot of) from the teacher essays simpler. In general, `statsmodels` is a useful library for traditional social science research, where we're interested in fitting a statistical model and then examining coefficients and looking at values like log likelihood to compare models. `sklearn` follows a tradition more closely linked to machine learning, where we're primarily interested in prediction and how accurate our model is in a prediction setting. Nevertheless, we'll still be able to use our results in the section below to produce a dataset similar to the one Mitra and Gilbert did in their paper.

## sklearn

We'll now get to the meat of the project where we'll extract text features from teacher essays and use those to predict the likelihood of a project being funded or not. First, we'll convert all the text to lowercase and take a quick peek to make sure that worked.

_Note:_ Columns in DataFrames are represented as pandas `Series`. `Series` that contain strings expose the `str` attribute which often allows for very elegant ways of handling string data.

In [22]:
all_projects.essay = all_projects.essay.str.lower()
all_projects.essay.head()

0    if they can't learn the way we teach, we teach...
1    which is bigger, three liters or three quarts?...
2    do you remember classrooms that used just book...
3    my class was given the beautiful gift of books...
4    thinking back in school science was either rea...
Name: essay, dtype: object

To begin, we'll focus on the trickiest pre-processing step. In the original paper only looked at phrases that appeared in all categories on Kickstarter. The purpose of this was to ensure that the phrases used were generalizable. From the original paper:

> Moreover, some phrases might be specific to certain project
categories. For example, the phrase _game credits_ has a considerable
presence in the entire corpus (140 occurrences), but
unsurprisingly appears most often in projects belonging to the
Games category. The phrase _our menu_ has a large presence
in the ‘Food’ category, but rarely appears in other categories.
Because of this, we need a way to guard against phrases that
uniquely identify categories and threaten generalizability. So,
we keep only those phrases which are present in all thirteen
categories. 

We'll attempt to do this by using only phrases that appear in each of the primary focus subjects in our dataset.

To do this, we'll use the `groupby` functionality to group all the essays from each primary focus subject together. We'll get the unigrams, bigrams, and trigrams from these documents, and only keep the ones that appear in all 27 of these documents. First, let's just create the documents. We'll group by `primary_focus_subject`, and then we'll apply a string join. The result is a list of combined essays for each focus subject.

In [23]:
grouped_by_subject_area = all_projects.groupby("primary_focus_subject")
essays = grouped_by_subject_area["essay"].apply(lambda x: ' '.join(x))
print essays

primary_focus_subject
Applied Sciences         kids love science time!  with very little reso...
Character Education      life is more fun if you play games."  roald da...
Civics & Government      i teach world history, law, and ap government....
College & Career Prep    how will i survive in the real world? my speci...
Community Service        in my classroom, we think, we ponder, we creat...
ESL                      one classroom computer with 22 students create...
Early Development        my vpk (voluntary pre-kindergarten) classroom ...
Economics                really, first graders learning economics? yes!...
Environmental Science    do you remember classrooms that used just book...
Extracurricular          chess can be alive always and rewarding !"  - ...
Foreign Languages        i have the amazing opportunity to help student...
Gym & Fitness            when can we go outside and have pe?"  "wow, i ...
Health & Life Science    outdated computers, overhead projectors, and v...
Hea

Now we'll use the `CountVectorizer` from `sklearn`. The purpose of the `CountVectorizer` is to get a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) representation from a set of documents. For now, we'll use it to set some options like `stop_words=stopwords.words('english')` to ignore the stopwords that ship with NLTK, `ngram_range=(1,3)` to specifiy we want unigrams, bigrams, and trigrams, and `min_df=27` to specify we want phrases that appear in every category (there are 27 focus subjects). We call the `fit_transform` method to get the `CountVectorizer` to run. Afterwards, it will have all the tokens in a dictionary in an attribute `vocabulary_`. We'll store these in a variable called terms.

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
vectorizer = CountVectorizer(stop_words=stopwords.words('english'), ngram_range=(1,3), min_df=27)
vectorizer.fit_transform(essays.tolist())
terms = vectorizer.vocabulary_.keys()

Next, we'll use the `CountVectorizer` from `sklearn` to get a bag-of-words representation of the teacher essays. We pass in our new `terms` variable to the `vocabulary` argument so we only use those words in our representation. We also set `min_df=50`, so that now we'll ignore any oh the phrases that don't occur in at least 50 documents, as Mitra and Gilbert did. Note this time when we call `fit_transform` we're storing the results in a variable called `word_features`. This function returns the matrix representation of our bag of words model.

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
vectorizer = CountVectorizer(ngram_range=(1,3), vocabulary=terms, min_df=50)
word_features = vectorizer.fit_transform(all_projects.essay)

Now we can get to actually running our classifier. This is as simple as importing `LogisticRegression` from `sklearn`. To begin, we'll also use the `cross_val_score` function from `sklearn`. This takes in a classifier, a dataset, and an argument `cv` and will running k-fold cross validation (where `k` is equal to the value passed as `cv`) and return accuracy scores. First we'll run just using our features from earlier, then we'll run using all our new text features.

_Note_: Notice we set `penalty="l1"` in the LogisticRegression model. This is the equivalent of using LASSO regression as Mitra and Gilbert did. Also note that our `X` and `y` variables are stored as as `pandas DataFrame` objects. `sklearn` expects `numpy` arrays. We can get these by calling `values` on the `DataFrame` objects, i.e., calling `X.values`. Also note the line `c, r = y.shape`. The short explanation of this is `sklearn` expects our `y` variable to be in a different shape than what is returned when we called `values` on the variable, so we need to call `reshape` for it to run correctly. If you're interested in this, you can read more [here](http://stackoverflow.com/questions/22053050/difference-between-numpy-array-shape-r-1-and-r).

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
classifier = LogisticRegression(penalty="l1")
c, r = y.shape
scores = cross_val_score(classifier, X.values, y.values.reshape(c,), cv=5, n_jobs=1)
print "Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)

Accuracy: 0.71 (+/- 0.01)


Now we'll add in our text features. To do this, it's useful to think about the data structures we're using with. We have our `word_features` matrix, where each row represents a document and each column represents a value in our bag of words model.

In [27]:
word_features.shape

(543693, 8953)

We also have our `X` variable from earlier with our other predictor variables.

In [28]:
X.values.shape

(543693, 31)

To combine these, we'll simply use the `hstack` function from the `scipy` library. This will just combine our two matrices into one.

_Note_: Notice we're importing `scipy.sparse`. The `word_features` matrix we got from the `CountVectorizer` is a [sparse matrix](https://en.wikipedia.org/wiki/Sparse_matrix). In short, bag of word representations will have a lot of 0's, since most documents don't have all our phrases. Storing these as a sparse matrix makes the representation is more efficient in memory use.

In [29]:
import scipy.sparse
all_features = scipy.sparse.hstack([word_features, X.values])
all_features.shape

(543693, 8984)

In [30]:
scores = cross_val_score(classifier, all_features, y.values.reshape(c,), cv=5, n_jobs=1)
print "Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)    

Accuracy: 0.71 (+/- 0.02)


Unlike in Mitra and Gilbert's paper, we don't see a decline in the error rate with the introduction of phrases from the essays. In other words, the predictive power of our model did not increase by including the word features. If our goal is just to build a better prediction model, we could experiment with other types of features and use the `cross_val_score` function to evaluate these different models. The nice thing about `sklearn` is it's easy to adjust arguments in functions to try out different features. If we wanted to use a bigger vocabulary, and not look at words and phrases that only appear in all of the subject focus areas, we could simply change the `min_df` argument in our first use of a `CountVectorizer` to build a vocabulary. If we want to only look at unigrams and bigrams, we can change the `ngram_range` to `(1,2)` rather than `(1,3)`. The `LogisticRegression` class also takes different arguments. We have a class imbalance in our dataset of funded and not funded projects. In other words, we have more funded projects than projects that aren't funded. If we wanted to automatically adjust weights in our model based on the proportion of classes, we could pass the argument `class_weight='balanced'` to the `LogisticRegression` class. `sklearn` gives you the option of trying different parameters to see what the best performing model is through [grid search](http://scikit-learn.org/stable/modules/grid_search.html). For now, I'll just focus on how to take the results of our current model and explore the phrases and export a dataset similar to the one Mitra and Gilbert published.

First, we'll call `classifier.fit`. This will simply fit a model using all our features. We can get the phrases we used from `CountVectorizer` by calling `get_feature_names()`.

To explore these phrases and which ones were most predictive of funded projects and which were predictive of non funded projects, we'll use the coefficients associated with each phrase in our model. These are accessible via the `coef_` attribute of our classifier model. To sort the coefficients, we'll use the `argsort` function from `numpy`. This function takes in an array and returns an array of the indices of the sorted values of the input array. For example, if we call `np.argsort(np.array([3, 1, 2])`, it would return `[1, 2, 0]`, because the lowest value has index 1 in the input array, the next highest value had index 2, and the highest value has index 0. By default, `argsort` will return values from lowest to highest. If we want to go from highest to lowest, we can reverse the array using the index `[::-1]`.

Now we can simply loop through the sorted coefficients and write them out to a tab separated file. I also print out the top 5 most predictive phrases for funded and non funded projects. Remember that not all of our features were our phrases, we also included variables such like `school_metro`. Since these variables won't have any variables in our `feature_names` variable, we'll use the condition statement `if coef_pos < word_features.shape[1]` before trying to write out any features. In other words, we make sure that the coefficient we're looking at is one of our word features and not one of the other variables before trying to look it up in our `feature_names` array. The resulting betas.csv file is available [in this repository](https://github.com/matthewheston/matthewheston.github.io/tree/master/etc/donorschoose).

In [31]:
import numpy as np

classifier.fit(all_features, y.values.reshape(c,))
feature_names = np.asarray(vectorizer.get_feature_names())
sorted_coefficient_positions = np.argsort(classifier.coef_[0])[::-1]
for coef_pos in sorted_coefficient_positions[:5]:
    if coef_pos < word_features.shape[1]:
        print feature_names[coef_pos], classifier.coef_[0][coef_pos]
for coef_pos in sorted_coefficient_positions[-5:]:
    if coef_pos < word_features.shape[1]:
        print feature_names[coef_pos], classifier.coef_[0][coef_pos]
with open("betas.csv", "w") as out_file:
    for coef_pos in sorted_coefficient_positions:
        if coef_pos < word_features.shape[1]:
            out_file.write("%s\t%s\n" % (feature_names[coef_pos], classifier.coef_[0][coef_pos]))

necessary success 7.37664862087
able purchase 1.54508320137
donation project 1.50476592355
things classroom 1.33749214308
ni teach school 1.33271849904
order learn -1.4306890903
goal project -1.50746100955
prepare college -1.6436739439
order become -1.67173392755
change world -1.8176379482


Recall that we removed stop words from the essays. So it looks like the phrase "neccesary for success" is most predictive of a project being funded while a phrase like "change the world" is most predictive of a project not being funded. Mitra and Gilbert discussed the phrases they found in terms of theories of persuasion. It may be interesting to compare the types of phrases we see in a corpus like DonorsChoose projects where teachers are asking for what may be thought of more as charitable giving rather than Kickstarter where projects may be promising something to those who support them.

To get a more generalizable list of phrases, Mitra and Gilbert compared their list to the [Google IT corpus](https://catalog.ldc.upenn.edu/ldc2006t13). They also grouped phrases using [LIWC](http://liwc.wpengine.com/). I won't include these steps in this notebook at this time, but those interested should be able to use the results of this notebook to do these analyses as well.

If anything on this page doesn't make sense, or you find any errors in code, please get in touch. My contact information is available [on my webpage](http://matthewheston.com).